In [1]:
!ls -a

'ls' は、内部コマンドまたは外部コマンド、
操作可能なプログラムまたはバッチ ファイルとして認識されていません。


In [2]:
import pandas as pd
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

import utils.read_data as rd
import utils.io_model as io_m
import utils.preprocessing as pp

In [3]:
import os
from os.path import join, dirname
from dotenv import load_dotenv
from pathlib import Path

In [4]:
load_dotenv(verbose=True)
dotenv_path = join(Path().resolve(), '.env')
load_dotenv(dotenv_path)

True

In [5]:
GOOGLE_DRIVE_PATH = os.environ.get("GOOGLE_DRIVE_PATH") + '/horse_racing'
DATA_PATH = GOOGLE_DRIVE_PATH + '/csv/'

In [6]:
print(GOOGLE_DRIVE_PATH)

C:\Users\endo_\Dropbox\workspace\HorseRacing/horse_racing


In [7]:
df = rd.read_horse_csv(DATA_PATH)

c:\users\endo_\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\users\endo_\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (0,16) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
c:\users\endo_\anaconda3\envs\py37\lib\site-packages\IPython\core\interactiveshell.py:3338: DtypeWarning: Columns (16) have mixed types.Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 639818 entries, 0 to 639817
Data columns (total 19 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   race_id             639818 non-null  float64
 1   rank                639818 non-null  object 
 2   frame_number        639818 non-null  int8   
 3   horse_number        639818 non-null  int8   
 4   horse_id            639818 non-null  int32  
 5   sex_and_age         639818 non-null  object 
 6   burden_weight       639818 non-null  float16
 7   rider_id            639818 non-null  int32  
 8   goal_time           634285 non-null  object 
 9   goal_time_dif       589557 non-null  object 
 10  half_way_rank       636449 non-null  object 
 11  last_time           634280 non-null  float16
 12  odds                639818 non-null  object 
 13  popular             637487 non-null  float16
 14  horse_weight        639818 non-null  object 
 15  tamer_id            639818 non-nul

In [9]:
df.head()

,race_id,rank,frame_number,horse_number,horse_id,sex_and_age,burden_weight,rider_id,goal_time,goal_time_dif,half_way_rank,last_time,odds,popular,horse_weight,tamer_id,owner_id,prize,total_horse_number
0,2.008060e+11,1,1,2,2005102028,牝3,54.0,733,1:13.9,NaN,1-1,39.18750,8.1,4.0,450(-10),138,498800,500.0,16
1,2.008060e+11,2,3,5,2005101044,牝3,54.0,663,1:13.9,ハナ,10-7,38.50000,13.4,6.0,482(+2),1088,163002,200.0,16
2,2.008060e+11,3,2,3,2005104156,牝3,54.0,684,1:14.2,1.3/4,13-8,38.50000,8.2,5.0,458(+4),1054,486800,130.0,16
3,2.008060e+11,4,4,7,2005103612,牝3,54.0,1095,1:14.3,3/4,2-2,39.50000,2.9,1.0,456(-2),436,274002,75.0,16
4,2.008060e+11,5,6,12,2005102035,牝3,54.0,1077,1:14.5,1.1/4,3-3,39.59375,4.3,2.0,458(-10),1007,498800,50.0,16


In [ ]:
df

## 前処理

In [10]:
# 学習に使用するカラム、過去データは3レース前までのデータを用いる
# これらのカラムに対して加工を行う為、新たなカラムが加わる。
# その為、最終的に用いる特徴量のカラムは別なセルで定義する(input_columns)
columns = ["race_course", "weather", "ground_status", 
           "where_racecourse", "race_class", "running_condition", 
           "frame_number", "horse_number",
           "sex_and_age", "burden_weight", "rider_id", 
           "tamer_id", "horse_weight", "odds", "popular",
           "rank", "total_horse_number_x", 
           "rank-1", "rank-2", "rank-3",
           "total_horse_number_x-1", "total_horse_number_x-2","total_horse_number_x-3",
           "goal_time-1", "goal_time-2", "goal_time-3",
           "last_time-1", "last_time-2", "last_time-3", 
           "half_way_rank-1", "half_way_rank-2", "half_way_rank-3", 
           "prize-1", "prize-2", "prize-3"]

df = df[columns]
df.head()

KeyError: "['where_racecourse', 'total_horse_number_x-1', 'race_class', 'goal_time-1', 'half_way_rank-2', 'rank-2', 'half_way_rank-1', 'prize-3', 'ground_status', 'prize-2', 'total_horse_number_x', 'running_condition', 'total_horse_number_x-3', 'goal_time-2', 'goal_time-3', 'last_time-2', 'rank-1', 'half_way_rank-3', 'last_time-1', 'rank-3', 'weather', 'last_time-3', 'race_course', 'prize-1', 'total_horse_number_x-2'] not in index"

In [ ]:
df["where_racecourse"] = df["where_racecourse"].map(pp.extract_place)

df["sex"] = df["sex_and_age"].map(lambda sex_and_age: sex_and_age[0])
df["age"] = df["sex_and_age"].map(lambda sex_and_age: sex_and_age[1:])

df["goal_time-1"] = df["goal_time-1"].map(pp.to_seconds)
df["goal_time-2"] = df["goal_time-2"].map(pp.to_seconds)
df["goal_time-3"] = df["goal_time-3"].map(pp.to_seconds)

df["horse_weight"] = df["horse_weight"].map(pp.extract_weight).astype(np.int64)

df["prize-1"] = df["prize-1"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)
df["prize-2"] = df["prize-2"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)
df["prize-3"] = df["prize-3"].map(lambda prize: prize.replace(",", "") if type(prize) == str else prize).astype(np.float64)

df["kyakusitu-1"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df["total_horse_number_x-1"].values, df["half_way_rank-1"])]
df["kyakusitu-2"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df["total_horse_number_x-2"].values, df["half_way_rank-2"])]
df["kyakusitu-3"] = [pp.kyakusitu_code_c(n, r) for n, r in zip(df["total_horse_number_x-3"].values, df["half_way_rank-3"])]

# 欠損値処理
df = df.replace('---', -1)
df = df.fillna(-1)

df["odds"] = df["odds"].astype(np.float64)

In [ ]:
# 欠損値の確認
df.isnull().sum().sum()

### ラベルの作成


In [ ]:
df["label"] = pp.make_label(df["rank"].values, df["total_horse_number_x"].values)
df["rank-1"] = pp.make_label(df["rank-1"].values, df["total_horse_number_x-1"].values)
df["rank-2"] = pp.make_label(df["rank-2"].values, df["total_horse_number_x-2"].values)
df["rank-3"] = pp.make_label(df["rank-3"].values, df["total_horse_number_x-3"].values)

In [ ]:
input_columns = ["race_course", "weather", "ground_status", 
                 "where_racecourse", "race_class", "running_condition", 
                 "frame_number", "horse_number",
                 "sex", "age", "burden_weight", "rider_id", 
                 "tamer_id", "horse_weight", "popular",
                 "rank-1", "rank-2", "rank-3", "odds", 
                 "goal_time-1", "goal_time-2", "goal_time-3",
                 "last_time-1", "last_time-2", "last_time-3", 
                 "kyakusitu-1", "kyakusitu-2", "kyakusitu-3", 
                 "prize-1", "prize-2", "prize-3", "label"]

In [ ]:
# one-hot
df_one_hot_encoded = pp.one_hot_encoding(df[input_columns])

In [ ]:
# 学習に使う特徴量名リスト
features = df_one_hot_encoded.columns.values.tolist()
features.remove("label_high")
features.remove("label_middle")
features.remove("label_low")

## 学習

In [ ]:
# 学習に用いるデータセットの作成
x = np.array(df_one_hot_encoded[features])
y = np.array(df_one_hot_encoded[["label_high", "label_middle", "label_low"]])
#del df
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=11)
#del x, y
print(x_train.shape)
print(x_test.shape)

In [ ]:
# データセットのシャッフルとバッチ化
train_ds = tf.data.Dataset.from_tensor_slices(
    (x_train, y_train)).shuffle(10000).batch(1024)

test_ds = tf.data.Dataset.from_tensor_slices((x_test, y_test)).batch(1024)

In [ ]:
import utils.sample_model as model
    
# モデルのインスタンスを作成
model = model.HorseModel(x_train.shape[1], 3)

In [ ]:
metrics = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.AUC(name='auc'),
]
model.compile(optimizer='adam', 
              loss='categorical_crossentropy',
              metrics=metrics)

In [ ]:
model.fit(train_ds, epochs=100)

In [ ]:
model.evaluate(test_ds, verbose=2)

In [ ]:
#テストデータの予測値と正解ラベルの確認
for x, y in zip(x_test, y_test):
    print(f"pred: {model.predict(x.reshape(1, -1))}, label: {y}")

In [ ]:
# モデルの保存
# io_m.save_model(model, model_name="first_model")

In [ ]:
# 保存したモデルに不具合がないか確認
# model = io_m.read_model("first_model")
# model.evaluate(test_ds, verbose=2)